### ***Faiss***

***Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.***

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [5]:
loader = TextLoader("data/speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [6]:
docs

[Document(metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'data/speech.txt'}, page_content='…\n\nIt will be all the easier for us to c

In [9]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
db = FAISS.from_documents(docs, embeddings)
db

#### ***Querying***

In [10]:
### querying 
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### ***As a Retriever***
***We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers***

In [11]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### ***Similarity Search with score***
***There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance (Manhattan Distance). Therefore, a lower score is better.***

In [12]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='585c9484-8031-4fd6-91b0-01a60b7d95cc', metadata={'source': 'data/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(204.16031)),
 (Document(id='3ed60149-69c0-4ad7-bdcc-b05d73c75c41', metadata={'source':

In [13]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.5108399391174316,
 -0.39273127913475037,
 -0.2813504934310913,
 0.44790032505989075,
 -0.7403210401535034,
 -1.5125731229782104,
 -0.7015759944915771,
 0.33812832832336426,
 0.04696910083293915,
 -0.1354532241821289,
 0.28600990772247314,
 0.052725955843925476,
 -0.33675435185432434,
 0.5880119800567627,
 -0.3939228057861328,
 -0.41346099972724915,
 -0.7215784192085266,
 -0.20970992743968964,
 0.25604262948036194,
 0.04926772788167,
 -0.03386495262384415,
 -0.4083031713962555,
 -0.6066664457321167,
 0.3827327489852905,
 -0.11274579912424088,
 0.27389246225357056,
 -0.5984042286872864,
 0.2036181539297104,
 1.1325715780258179,
 -0.11250502616167068,
 -0.17277905344963074,
 0.5429063439369202,
 -0.04088418185710907,
 -0.6854432225227356,
 0.4343262314796448,
 -0.6311648488044739,
 0.4278862774372101,
 -1.9120938777923584,
 0.13503752648830414,
 -0.15908676385879517,
 0.25243258476257324,
 0.28920188546180725,
 0.557979941368103,
 -0.4555892050266266,
 -1.473099946975708,
 -0.282497316

In [14]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='585c9484-8031-4fd6-91b0-01a60b7d95cc', metadata={'source': 'data/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='3ed60149-69c0-4ad7-bdcc-b05d73c75c41', metadata={'source': 'data/speech.txt'}, page_co

In [15]:
### Saving And Loading
db.save_local("vectordb/faiss_index")

In [16]:
new_db=FAISS.load_local("vectordb/faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(id='585c9484-8031-4fd6-91b0-01a60b7d95cc', metadata={'source': 'data/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='3ed60149-69c0-4ad7-bdcc-b05d73c75c41', metadata={'source': 'data/speech.txt'}, page_co